In [16]:
# reading in all necessary libraries
import networkx as nx
import os
import pandas as pd
os.chdir('/home/Nickfis/Documents/Projects/football_network_analysis')
import numpy as np
%matplotlib inline

## reading in data

the dataframes that are loaded in here were derived earlier from the overall event data for the season
plus qualifiers that explain each event more precisely. 
This means the possession as well as the xg_features data is based on multiple assumptions made by me.
The possessions df gives each offensive possession in the season that ended in a shot. An offensive possession
is just defined as a sequence of passes from only one team that ended in a shot.
XG_features is basically a dataframe with about 16.000 rows of shots, with certain features that describe the
shot more closely, in order to obtain a probability for that shot going in. In this project, we only need to 
look at the probability of the shot going in, since this will give us a measure of danger of that offensive
possession. 

In [2]:
# event data: all events that occurred in every game of LaLiga over the season 2017/2018
event = pd.read_csv('event2017.csv')
# possession dataframes. All offensive possessions that ended in a shot
possession = pd.read_csv('possession2017.csv')
# expected goals 
xg_features = pd.read_csv('xg_features2017.csv')
# team and playerspecific information
teams = pd.read_csv('teams.csv')
allplayers = pd.read_csv('player_frame.csv')


In [9]:
# possession shape needs to be a little bit preprocessed.
# drop the duplicates
possession = possession[~possession.duplicated(keep='first')]
# leave out na's because not part of possession and not a player
possession = possession[~possession['player_id'].isna()]
# only look at shots and passes
shotsandpass = [1,13,14,15,16]
possession = possession[possession['event_type'].isin(shotsandpass)]
possession.index = possession['possession']
# deleting all possessions with fewer than 3 actions
possession = possession[pd.DataFrame(possession['possession'].value_counts())['possession']>2]

/home/Nickfis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:11: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # This is added back by InteractiveShellApp.init_path()


We will see that our y and x axis both range from 0 to 100. In reality though, the pitch is obviously not a square. Here I took a ratio, that should transform the y-coordinates in an adequate manner. 

In [13]:
# check for the y-coordinate
possession['y'].describe()

In [8]:
ratio=105/69.5

possession['y'] = possession['y']/ratio

/home/Nickfis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


## Building Adjacency Matrices for each team


In [14]:
#create adjacency matrix for each team
for team in possession['team_id'].unique():
    team_pos = possession[possession['team_id']==team]
    # create adjacency matrix for that team
    team_player = team_pos['player_id'].unique()
    # player x player matrix
    adj_matrix = pd.DataFrame(np.zeros((len(team_player),len(team_player))))
    # name indices
    adj_matrix.columns = team_player
    adj_matrix.index = team_player
    # loop through every possession for the team and add received / played passes for each player
    for pos in team_pos['possession'].unique():
        one_pos = team_pos[team_pos['possession']==pos]
        #print(pos)
        # now go back more and more in the dataframe till the beginning
        # first pass
        for k in range((one_pos.shape[0]-1),0,-1):
            #print(k)
            # player that passes the ball
            # player that passes
            i = one_pos.iloc[k]['player_id']
            #print(i)
            # player that receives the pass
            j = one_pos.iloc[k-1]['player_id']
            # filling up the values
            adj_matrix.loc[i,j]+=1

    ############################################ BUILDING POSITION DATAFRAME #########################################################
    # in another dataframe for that team I want to have all the players and their median (mean?) position and their number of actions
    # a df with 3 columns with the length of the players in the adjacency matrix, maybe even do a density of each player?
    team_positions = pd.DataFrame(team_player)
    team_positions.index = team_player
    # add positions
    team_positions['x'] = 0
    team_positions['y'] = 0
    # actions
    team_positions['actions'] = 0

    for i in team_player:
        #player_pos = team_pos[team_pos['player_id']==i]
        team_positions.loc[i,'x'] = team_pos[team_pos['player_id']==i]['x'].mean()
        team_positions.loc[i,'y'] = team_pos[team_pos['player_id']==i]['y'].mean()
        team_positions.loc[i, 'actions'] = len(team_pos[team_pos['player_id']==i]['possession'])


    team_positions.to_csv('positions_'+str(team)+'.csv')
    adj_matrix.to_csv('adj_matrix_'+str(team)+'.csv')

HELLO
HELLO
HELLO
HELLO
HELLO
HELLO
HELLO
HELLO
HELLO
HELLO
HELLO


8